## sqlmodel in in LinkML?

[sqlmodel](https://github.com/tiangolo/sqlmodel) is a library that bridges classes from SQLAlchemy and Pydantic. The idea is that with the most pydantic-like classes, you can also work with a database.

I think it would be nice to have a generator to produce sqlmodel code.

Let's take the simple LinkML schema below (full in [dummy.yaml](./dummy.yaml))

```yaml
slots:
  id:
    identifier: true
    slot_uri: schema:identifier
    range: integer
    description: A unique identifier for a thing
    required: true
  name:
    slot_uri: schema:name
    description: A human-readable name for a thing
    range: string
    required: true

classes:
  Person:
    slots:
      - id
      - name
    attributes:
      - age:
          range: integer
          description: The age of the person
      - team_id:
          range: Team
          description: The team to which the person belongs

    description: A person
  Team:
    slots:
      - id
      - name
    description: A team
```

The pydantic generator would make this (full output in [dummy.py](./dummy.py))

```python
class Person(ConfiguredBaseModel):
    """
    A person
    """
    id: int = Field(..., description="""A unique identifier for a thing""")
    name: str = Field(..., description="""A human-readable name for a thing""")
    age: Optional[int] = Field(None, description="""The age of the person""")
    team_id: Optional[int] = Field(None, description="""The team to which the person belongs""")


class Team(ConfiguredBaseModel):
    """
    A team
    """
    id: int = Field(..., description="""A unique identifier for a thing""")
    name: str = Field(..., description="""A human-readable name for a thing""")
```


The equivalent thing in sqlmodel would be something like the code below. This can't be ran in jupyter as far as I can tell, because the typing magic does not work. In this example, you can find the models in [sql_models.py](./sql_models.py).

```python
from sqlmodel import SQLModel, Relationship, Field
from typing import Optional


class TeamSQL(SQLModel, table=True):
    # primary_key is always optional in sqlmodel
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(None)

    people: list["PersonSQL"] = Relationship(back_populates="team")


class PersonSQL(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(None)
    age: int = Field(None)

    team_id: Optional[int] = Field(None, foreign_key="TeamSQL.id")
    team: "TeamSQL" = Relationship(back_populates="people")
```

This is pretty convenient because `SQLmodel`s are both pydantic classes (can be used with FastAPI) and SQLAlchemy objects, below we initialize the database and import the models (db initialization does not work on jupyter either).

The SQL is printed to the console

In [2]:
from sql_models import TeamSQL, PersonSQL, engine



BEGIN (implicit)
PRAGMA main.table_info("teamsql")
[raw sql] ()
PRAGMA temp.table_info("teamsql")
[raw sql] ()
PRAGMA main.table_info("personsql")
[raw sql] ()
PRAGMA temp.table_info("personsql")
[raw sql] ()

CREATE TABLE teamsql (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


[no key 0.00028s] ()

CREATE TABLE personsql (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	age INTEGER NOT NULL, 
	team_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(team_id) REFERENCES teamsql (id)
)


[no key 0.00023s] ()
COMMIT



The sql generated by sqlmodel in the cell above when creating the database is equivalent to the one generated with

```bash
gen-sqltables --dialect sqlite dummy.yaml > dummy.sql
```

Now let's add something to the db (the SQL is printed to the console)

In [3]:
from sqlmodel import Session

with Session(engine) as session:

    team_red = TeamSQL(name="Red")
    new_person = PersonSQL(name="Alice", age=30, team = team_red)
    session.add(new_person)
    session.commit()

    # We can access the fields:
    print()
    print('Fields:')
    print(new_person.id)
    print(new_person.name)

    # To access the entire model, we have to refresh it:
    print()
    session.refresh(new_person)
    print('Model:')
    print(new_person)


BEGIN (implicit)
INSERT INTO teamsql (name) VALUES (?)
[generated in 0.00044s] ('Red',)
INSERT INTO personsql (name, age, team_id) VALUES (?, ?, ?)
[generated in 0.00020s] ('Alice', 30, 1)
COMMIT
BEGIN (implicit)
SELECT personsql.id AS personsql_id, personsql.name AS personsql_name, personsql.age AS personsql_age, personsql.team_id AS personsql_team_id 
FROM personsql 
WHERE personsql.id = ?
[generated in 0.00030s] (1,)
SELECT personsql.id, personsql.name, personsql.age, personsql.team_id 
FROM personsql 
WHERE personsql.id = ?
[generated in 0.00018s] (1,)
ROLLBACK



Fields:
1
Alice

Model:
name='Alice' age=30 team_id=1 id=1


## If only it was that simple

Well, that's pretty easy, isnt' it?

The thing is that if you want to use a CRUD API with FastAPI, you have to define different models for the same thing. The recommended approach for a class like person is something like this instead ( the cell will give errors in jupyter)

Not sure if a generator with so many derived classes is without the scope of LinkML

In [4]:
from sqlmodel import Relationship, Field, SQLModel
from typing import Optional

# table=False means that a table will not be created for this model in the database
# this would be the equivalent of a pydantic model without relationship fields
class PersonSQLBase(SQLModel, table=False):
    name: str
    age: int


# This is the actual model that will be used in the database
class PersonSQL(PersonSQLBase, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)

    team_id: Optional[int] = Field(None, foreign_key="TeamSQL.id")
    team: "TeamSQL" = Relationship(back_populates="people")


class personCreate(PersonSQLBase):
    team_id: Optional[int]

class PersonRead(PersonSQLBase):
    id: int # not optional

# Everything is optional since we are updating.
# We can update relationships as well
class PersonUpdate(PersonSQLBase):
    name: Optional[str]
    age: Optional[int]
    team_id: Optional[int]

